In [1]:
import os
import glob
import json
from dateutil import parser
import pandas as pd
import numpy as np

In [2]:
this_path = './sessions_event_2'

In [3]:
files = [os.path.basename(x) for x in glob.glob('%s/*.json'%this_path)]

In [4]:
dt = pd.DataFrame()
for file_name in files[:1000]:
    f = open('%s/%s'%(this_path, file_name))
    context = f.readlines()
    data = []
    for i in context[:]:
        data += [json.loads(i.replace(',\n','').replace('\n',''))]
        
    uIds = []
    uIds_ind = []
    fPs = []
    fPs_ind = []
    ips = []
    ips_ind = []
    #print files.index(file_name)
    for field_name, list_mass, ind_mass in [('userId', uIds, uIds_ind), ('fingerPrint', fPs, fPs_ind), ('ip', ips, ips_ind)]:
        try:
            list_mass += [i[field_name] for i in data]
            temp_set = list(set(list_mass))
            if len(temp_set)>1:
                ind_mass += sorted([list_mass.index(i) for i in temp_set])
            else:
                ind_mass += [None]
        except KeyError:
            list_mass += [None]
            ind_mass += ['empty']
            pass
    deltas = [0]
    times = [parser.parse(i['timestamp']) for i in data]
    deltas += [(times[i] - times[i-1]).total_seconds() for i in range(1, len(times))]
    dt=dt.append({
            'UserIds': len(set(uIds)),
            'UserIds_ind': uIds_ind,
            'fingerPrint': len(set(fPs)),
            'fingerPrint_ind': fPs_ind,
            'ips': len(set(ips)),
            'ips_ind': ips_ind,
            'deltas': deltas
        }, ignore_index=True)
    print len(dt)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [22]:
for i in range(0, len(dt)):
    if max(dt.iloc[i]['deltas'])>3600:
        print i

4
52
157
183
201
207
310
375
416
594
624
662
818
872


In [24]:
dt.iloc[52]['deltas']

[0, 17.6, 97.711, 19.791, 195.325, 19.508, 5962942.202, 63.936, 58.758, 27.803]

In [25]:
f = open('%s/%s'%(this_path, files[52]))
print files[52]
for i in f.readlines():
    print i
    print 

5cc99be1-0645-4c21-860b-73e0ad4d170f, session_1.json
{"event":null,"fingerPrint":"522d7d4f812e1528434eb9cb7b89245d","ip":"100.37.251.113","path":"\/clinics\/country=lithuania\/illness=pancreatic-cancer\/","properties":null,"referrer":"https:\/\/www.google.com\/","search":"","timestamp":"2016-06-14T23:26:00.313+0000","type":1,"userId":null}


{"event":null,"fingerPrint":"522d7d4f812e1528434eb9cb7b89245d","ip":"100.37.251.113","path":"\/clinics\/country=litva\/illness=rak-podzheludochnoj\/","properties":null,"referrer":"\/clinics\/country=lithuania\/illness=pancreatic-cancer\/","search":"","timestamp":"2016-06-14T23:26:17.913+0000","type":1,"userId":null}


{"event":1.0,"fingerPrint":"522d7d4f812e1528434eb9cb7b89245d","ip":"100.37.251.113","path":"\/clinics\/country=litva\/illness=rak-podzheludochnoj\/","properties":{"button_type":"auto_help"},"referrer":"\/clinics\/country=lithuania\/illness=pancreatic-cancer\/","search":"","timestamp":"2016-06-14T23:27:55.624+0000","type":2,"userId":nu

In [9]:
files[35]

'91966d68-0670-48d7-9d4a-bb80914e3684, session_1.json'

In [ ]:
#reg_users
f = open('files_reg_users.txt', 'w')
ar1 = dt[dt['ips']==1][dt['fingerPrint'] ==1][dt['UserIds']==2].index
for i in ar1:
    f.write(files[i]+'\n')
f.close()

In [ ]:
hours = 12
f = open('files_slicable_users_by_ip.txt', 'w')
ar2 = []
for k in dt[dt['ips']>1][dt['fingerPrint']==1].index.values:
    inds = sorted([i for i in dt.iloc[k]['ips_ind'] if i !=0])
    flag = True
    for i in inds:
        if not abs(dt.iloc[k]['deltas'][i]) > 3600*hours:
            flag = False
            break
    if flag:
        #print k
        ar2 += [k]
        f.write(files[k]+'\n')
print len(ar2)
f.close()

In [ ]:
hours = 12
f = open('files_slicable_users_by_fP.txt', 'w')
ar3 = []
for k in dt[dt['ips']==1][dt['fingerPrint']>1].index.values:
    inds = sorted([i for i in dt.iloc[k]['fingerPrint_ind'] if i !=0])
    if inds == [None]:
        continue
    flag = True
    for i in inds:
        if not abs(dt.iloc[k]['deltas'][i]) > 3600*hours:
            flag = False
            break
    if flag:
        #print k
        ar3 += [k]
        f.write(files[k]+'\n')
print len(ar3)
f.close()

In [ ]:
total = sorted(list(set(list(ar1)+ar2+ar3)))

In [ ]:
f=open('files_unun_users.txt','w')
for i in range(0,len(files)):
    if i not in total:
        f.write(files[i]+'\n')
f.close()

In [ ]:
dt.to_csv('46000.csv')